In [156]:
import spacy
import pandas
import json

import random
import typer
from pathlib import Path
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example

# make the factory work
from rel_pipe import make_relation_extractor, score_relations

# make the config work
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

import custom_functions

In [157]:
# --> Loading Models
ner = spacy.load("/Users/leonidas/OneDrive - Universität St.Gallen/General/02-Coding/03-Models/NER trained on nightly/ner-cb1-159-15-12")
rel = spacy.load("/Users/leonidas/OneDrive - Universität St.Gallen/General/02-Coding/03-Models/Relations/0-trained-on-djx/2021-09-01/training/model-best")

In [158]:
# --> Load test data and name annotated data "golds"
test_data = "/Users/leonidas/ReciParse_Scripts/06_annotated_data_analysis/test.spacy"
doc_bin = DocBin(store_user_data=True).from_disk(test_data)
golds = doc_bin.get_docs(rel.vocab)

In [159]:
examples = []
def create_examples(predict_ents=True, print_details=False):
    for gold in golds:
        # ner prediction
        if predict_ents == True:
            pred = ner(gold.text)
        else:
            pred = Doc(rel.vocab,
                       words=[t.text for t in gold],
                       spaces=[t.whitespace_ for t in gold],)
            pred.ents = gold.ents
            
        # rel prediction
        for name, proc in rel.pipeline:
            pred = proc(pred)
        # Create pair of pred and anno & Append pair to list
        examples.append(Example(pred, gold))

        # Print comparison
        # print("--- TEXT ---")
        # print(gold.text)
        # print("--- GOLD ---")
        # print(example.reference.ents)
        # print("--- PRED ---")
        # print(example.predicted.ents)
        
        # print(pred._.rel.keys())
        # print(gold._.rel.keys())
        # break

        if print_details:
            print()
            print(f"Text: {gold.text}")
            print(f"spans: {[(e.start, e.text, e.label_) for e in pred.ents]}")
            for value, rel_dict in pred._.rel.items():
                gold_labels = [k for (k, v) in gold._.rel[value].items() if v == 1.0]
                if gold_labels:
                    print(
                        f" pair: {value} --> gold labels: {gold_labels} --> predicted values: {rel_dict}"
                    )
            print()
create_examples(False, True)

ℹ Detected 282 total instances.

Text: Knoblauch, Zitronengras, Ingwer, Peperoni, Fischsoße, 3 EL Zitronensaft und Zucker verrühren. Fleisch in ca. 1 cm große Würfel schneiden, mit der Hälfte der Marinade mischen.Glasnudeln mit kochendem Wasser übergießen, 5 Min. ziehen lassen, danach gut abtropfen. Mit Sesamöl und 1 EL Zitronensaft mischen.Möhre schälen, in feine Streifen raspeln. Gurke waschen, längs halbieren, entkernen und quer in dünne Scheiben schneiden. Gemüse mit den Glasnudeln mischen, Frühlingszwiebeln unter die restliche Marinade rühren.Fleisch auf Holzspieße stecken. Erdnussöl in einer Pfanne erhitzen, die Spieße darin ca. 7 Min. braten, dabei einmal wenden. Mit Glasnudelsalat und Frühlingszwiebelsoße anrichten.
spans: [(0, 'Knoblauch', 'Z'), (2, 'Zitronengras', 'Z'), (4, 'Ingwer', 'Z'), (6, 'Peperoni', 'Z'), (8, 'Fischsoße', 'Z'), (10, '3 EL Zitronensaft', 'Z'), (14, 'Zucker', 'Z'), (15, 'verrühren', 'V'), (17, 'Fleisch', 'Z'), (18, 'in ca. 1 cm große Würfel', 'ATTR'), (24

KeyError: (24, 0)

In [151]:
from spacy.scorer import PRFScore

def score_relations(examples, threshold):
    """Score a batch of examples."""
    micro_prf = PRFScore()
    for example in examples:
        gold = example.reference._.rel
        pred = example.predicted._.rel
        for key, pred_dict in pred.items():
            gold_labels = [k for (k, v) in gold[key].items() if v == 1.0]
            for k, v in pred_dict.items():
                if v >= threshold:
                    if k in gold_labels:
                        micro_prf.tp += 1
                    else:
                        micro_prf.fp += 1
                else:
                    if k in gold_labels:
                        micro_prf.fn += 1
    return {
        "rel_micro_p": micro_prf.precision,
        "rel_micro_r": micro_prf.recall,
        "rel_micro_f": micro_prf.fscore,
    }


threshold = 0
score_relations(examples, threshold)


{'rel_micro_p': 0.0, 'rel_micro_r': 0.0, 'rel_micro_f': 0.0}

In [66]:
for example in examples:
    gold = example.reference._.rel
    pred = example.predicted._.rel
    for key, pred_dict in pred.items():
        print(item)
        gold_label = list(gold[key].keys())[list(gold[key].values()).index(1)]
        print(gold_label)
        
    break

((4, 1), {'Arg0Z': 0.98250717, 'Arg0Tool': 0.0027200435, 'Arg1Z': 2.4258585e-05, 'Arg1Tool': 0.00021370326, 'ArgAttr': 0.0058646654, 'ArgTemp': 0.0008637497, 'ArgDauer': 0.00029359866, 'ArgZeitp': 0.00420872, 'ArgPräp': 1.6493268e-05})
Arg0Z
((4, 1), {'Arg0Z': 0.98250717, 'Arg0Tool': 0.0027200435, 'Arg1Z': 2.4258585e-05, 'Arg1Tool': 0.00021370326, 'ArgAttr': 0.0058646654, 'ArgTemp': 0.0008637497, 'ArgDauer': 0.00029359866, 'ArgZeitp': 0.00420872, 'ArgPräp': 1.6493268e-05})
ArgAttr
((4, 1), {'Arg0Z': 0.98250717, 'Arg0Tool': 0.0027200435, 'Arg1Z': 2.4258585e-05, 'Arg1Tool': 0.00021370326, 'ArgAttr': 0.0058646654, 'ArgTemp': 0.0008637497, 'ArgDauer': 0.00029359866, 'ArgZeitp': 0.00420872, 'ArgPräp': 1.6493268e-05})


ValueError: 1 is not in list

In [11]:
relation_extractor = rel.get_pipe("relation_extractor")
get_instances = relation_extractor.model.attrs["get_instances"]

In [12]:
# --> Create Random Baseline
rands = preds.copy()
# Loop through instances in one recipe
for pred in rands:
    for (e1, e2) in get_instances(pred):
        # Create offset tuple: (anbraten, Würste) --> (2,1)
        offset = (e1.start, e2.start)
        # Check whether offset is a key in relations 
        if offset not in pred._.rel:
            # Create an empty dictionary for every insatnce
            pred._.rel[offset] = {}
        for label in relation_extractor.labels:
            # Assign random probability to each label
            pred._.rel[offset][label] = random.uniform(0, 1)

In [19]:
def _score_and_format(examples, thresholds):
    for threshold in thresholds:
        r = score_relations(examples, threshold)
        results = {k: "{:.2f}".format(v * 100) for k, v in r.items()}
        print(f"threshold {'{:.2f}'.format(threshold)} \t {results}")

thresholds = [0.000, 0.050, 0.100, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
examples = [Example(pred, gold)]
_score_and_format(examples, thresholds)

threshold 0.00 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.05 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.10 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.20 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.30 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.40 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.50 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.60 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.70 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.80 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.90 	 {'rel_micro_p': '0.00', 'rel_micro_r': '0.00', 'rel_micro_f': '0.00'}
threshold 0.99 	 {'rel_micro_p': '0.00', 'r